In [1]:
from symred.symplectic_form import PauliwordOp, StabilizerOp
from symred.S3_projection import (S3_projection, 
                                  gf2_gaus_elim, 
                                  gf2_basis_for_gf2_rref, 
                                  QubitTapering, 
                                  CS_VQE, 
                                  CheatS_VQE)
from symred.build_model import build_molecule_for_projection
from symred.utils import exact_gs_energy, quasi_model
import json
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import openfermion as of
import openfermionpyscf as ofpyscf
from openfermion.circuits import ( uccsd_singlet_get_packed_amplitudes,
                                   uccsd_singlet_generator, uccsd_generator,
                                   uccsd_convert_amplitude_format)
from cirq.testing import random_unitary
from itertools import combinations, product

import warnings
import os

In [2]:
cwd = os.getcwd()
symred_dir = os.path.dirname(cwd)
data_dir = os.path.join(symred_dir, 'data')
mol_list = os.listdir(data_dir)

In [3]:
mol_list

['H1-O1-Na1_STO-3G_singlet_NaOHNaOH.json',
 'C1-O2_STO-3G_singlet_CO2CO2.json',
 'H2-C1-O1_STO-3G_singlet_MethanalMethanal.json',
 'H6-C3_STO-3G_singlet_PropenePropene.json',
 'H2_3-21G_singlet_H2_3-21GH2_3-21G.json',
 'H6-C3-O1_STO-3G_singlet_PropanonePropanone.json',
 'H1-O1_STO-3G_singlet_-1-_OH-OH-.json',
 'H2_6-311G*_singlet_H2_6-311G*H2_6-311G*.json',
 'H1-Cl1_STO-3G_singlet_HclHcl.json',
 'H5-C2-N1-O1_STO-3G_singlet_EthanamideEthanamide.json',
 'H2-Mg1_STO-3G_singlet_MgH2MgH2.json',
 'Si1_STO-3G_triplet_SiSi.json',
 'Cl2_STO-3G_singlet_Cl2Cl2.json',
 'H2-S1_STO-3G_singlet_H2SH2S.json',
 'S1_STO-3G_triplet_SS.json',
 'Br1_STO-3G_doublet_BrBr.json',
 'H2_STO-3G_singlet_H2H2.json',
 'H1-He1_6-31+G**_singlet_1+_HeH+_6-31+G**HeH+_6-31+G**.json',
 'H1-Na1_STO-3G_singlet_NaHNaH.json',
 'F1_STO-3G_doublet_FF.json',
 'O1_STO-3G_triplet_OO.json',
 'H2-C1_STO-3G_triplet_CH2CH2.json',
 'H2-C2_STO-3G_singlet_EthyneEthyne.json',
 'N1-O3_STO-3G_singlet_-1-_NO3-NO3-.json',
 'O2_STO-3G_singlet_O

# Get Hamiltonian

In [4]:
speciesname = 'Be1_STO-3G_singlet_BeBe.json'
# speciesname = 'H2_STO-3G_singlet_H2H2.json',
mol_file = os.path.join(data_dir, speciesname)
with open(mol_file, 'r') as infile:
    ham = json.load(infile)

ham_op = PauliwordOp(ham)
# print(ham_op)

# Convert to qubits, taper and construct CS-VQE model

In [5]:
contextual_sub = CS_VQE(ham_op)

/Users/lex/anaconda3/envs/symred/lib/python3.9/site-packages/symred/S3_projection.py:302: ComplexWarning: Casting complex values to real discards the imaginary part
  ham_nc_dict = {op:float(coeff) for op,coeff in self.noncontextual_operator.to_dictionary.items()}


In [6]:
contextual_sub.clique_operator.coeff_vec

array([-1.00000000e+00, -1.02213201e-08])

In [7]:
contextual_sub.G_indices.shape

(74, 9)

In [8]:
contextual_sub.r_indices.shape

(74, 2)

In [9]:
contextual_sub.noncontextual_objective_function(contextual_sub.symmetry_generators.coeff_vec,
                                        contextual_sub.clique_operator.coeff_vec)

-14.351880476202037

In [10]:
contextual_sub.symmetry_generators.coeff_vec

array([-1,  1,  1,  1,  1,  1,  1,  1,  1])

In [11]:
# print(contextual_sub.noncontextual_operator)

In [12]:
# contextual_sub.noncontextual_operator

In [60]:
#### random vecs (allowed to be non ints for G!)
n_gen = contextual_sub.symmetry_generators.n_terms 
g_vec = np.random.rand(n_gen) * np.random.choice([-1,1], size=n_gen)
g_vec_norm = g_vec / np.linalg.norm(g_vec)

r_vec = np.random.rand(contextual_sub.n_cliques) * np.random.choice([-1,1], size=contextual_sub.n_cliques)


#### correct vecs
g_vec_norm = contextual_sub.symmetry_generators.coeff_vec
r_vec = contextual_sub.clique_operator.coeff_vec



r_exp = contextual_sub.r_indices * r_vec
r_exp[np.where(r_exp==0)]=1

g_exp = contextual_sub.G_indices * g_vec_norm
g_exp[np.where(g_exp==0)]=1 # set values not affecting cost to 1 (so np.prod doesn't go to zero!)

G = np.prod(g_exp, axis=1)
R = np.prod(r_exp, axis=1)

np.sum(contextual_sub.noncontextual_operator.coeff_vec*G*R).real



-14.351880476202037

In [403]:
def new_cost_F(vector):
    # vector = (q, r)
    
    if isinstance(vector, list):
        vector = np.array(vector)
    
    g_exp = contextual_sub.G_indices * vector[:contextual_sub.symmetry_generators.n_terms]
    g_exp[np.where(g_exp==0)]=1 # set values not affecting cost to 1 (so np.prod doesn't go to zero!)
    
    r_exp = contextual_sub.r_indices * vector[contextual_sub.symmetry_generators.n_terms:]
    r_exp[np.where(r_exp==0)]=1

    G = np.prod(g_exp, axis=1)
    R = np.prod(r_exp, axis=1)

#     return np.sum(contextual_sub.noncontextual_operator.coeff_vec.real*G*R)
    return np.dot(contextual_sub.noncontextual_operator.coeff_vec.real,  G*R)
    

In [393]:
size = contextual_sub.symmetry_generators.n_terms + contextual_sub.n_cliques 
random_vec = np.random.rand(size) * np.random.choice([-1,1], size=size)
random_vec = random_vec / np.linalg.norm(random_vec)
new_cost_F(random_vec)

-9.176020339519699

In [257]:
size = contextual_sub.symmetry_generators.n_terms + contextual_sub.n_cliques 
random_vec = np.random.rand(size) * np.random.choice([-1,1], size=size)

random_vec[contextual_sub.symmetry_generators.n_terms:]

array([-0.67980422,  0.34194459])

In [443]:
def unit_sphere(angles):
    if isinstance(angles, list):
        angles = np.array(angles)
        
    cartesians = [np.prod(np.sin(angles[:i]))*np.cos(angles[i]) for i in range(len(angles))]
    cartesians.append(np.prod(np.sin(angles)))
    vec = np.array(cartesians)
    return vec
    
def new_cost_F_rangle(vector):
    # vector = (q, r) = (q, angle)
    # r = (cos(angle), sin(angle))  
    # note vector here is size |G|+|C|-1
    
    if isinstance(vector, list):
        vector = np.array(vector)
    
    g_exp = contextual_sub.G_indices * vector[:contextual_sub.symmetry_generators.n_terms]
    g_exp[np.where(g_exp==0)]=1 # set values not affecting cost to 1 (so np.prod doesn't go to zero!)
    
    
#     angle = vector[-1]
#     r_vec = np.array([np.cos(angle),np.sin(angle)])
    angles = vector[contextual_sub.symmetry_generators.n_terms:]
    r_vec = unit_sphere(angles)
#     print(np.linalg.norm(r_vec))
    
    r_exp = contextual_sub.r_indices * r_vec
    r_exp[np.where(r_exp==0)]=1

    G = np.prod(g_exp, axis=1)
    R = np.prod(r_exp, axis=1)

#     return np.sum(contextual_sub.noncontextual_operator.coeff_vec*G*R).real
    return np.sum(contextual_sub.noncontextual_operator.coeff_vec.real*G*R)
    

In [444]:
from scipy.optimize import shgo, differential_evolution
size = contextual_sub.symmetry_generators.n_terms + contextual_sub.n_cliques -1

# G bounds
bounds = [(-1, +1)]* contextual_sub.symmetry_generators.n_terms

# R bounds
bounds += [(0, 2*np.pi)]* (contextual_sub.n_cliques -1)


# optimizer_output = shgo(func=new_cost_F, bounds=nu_bounds)

optimizer_output = differential_evolution(
                        func=new_cost_F_rangle, 
                        bounds=bounds,
                        maxiter=10_000) # limits how many runs (max iteration!)

print('minimum = ', optimizer_output.fun)
print('q_vec_opt = ', optimizer_output.x[:contextual_sub.symmetry_generators.n_terms])
print('r_vec_opt = ', unit_sphere(optimizer_output.x[contextual_sub.symmetry_generators.n_terms:]))
print('steps taken:', optimizer_output.nfev)

KeyboardInterrupt: 

In [442]:
from scipy.optimize import minimize

# random q_vec
G_size = contextual_sub.symmetry_generators.n_terms 
qvec = np.random.rand(G_size) * np.random.choice([-1,1], size=G_size)
# random r_vec
R_size = contextual_sub.n_cliques -1
rvec = np.random.uniform(low=0, high=2*np.pi, size=(R_size,))


# G bounds
bounds = [(-1, +1)]* contextual_sub.symmetry_generators.n_terms
# R bounds
bounds += [(0, 2*np.pi)]* (contextual_sub.n_cliques -1)

x0 = np.hstack((qvec, rvec))
## methods: TNC, SLSQP, L-BFGS-B
minimize(new_cost_F_rangle, x0, method='SLSQP', bounds=bounds)


     fun: 4.381813624780127e-07
     jac: array([-1.16093573e-06, -3.77041835e-06,  1.49335655e-06,  1.26570463e-05,
        7.34335252e-07, -6.43227494e-07,  8.66847564e-07,  2.26229675e-06,
       -2.04239775e-06,  5.17969418e-07])
 message: 'Optimization terminated successfully'
    nfev: 11
     nit: 1
    njev: 1
  status: 0
 success: True
       x: array([-0.37743812, -0.11621558,  0.29342046,  0.03461956,  0.59670449,
       -0.68122323,  0.50548846,  0.19368872, -0.21454261,  2.08929013])

In [320]:
contextual_sub.noncontextual_objective_function(contextual_sub.symmetry_generators.coeff_vec,
                                        contextual_sub.clique_operator.coeff_vec)

-14.351880476202037

In [376]:
b_lb

array([-1., -1., -1., -1., -1., -1., -1., -1., -1.,  0.])

In [ ]:
def unit_sphere(angles):
    if isinstance(angles, list):
        angles = np.array(angles)
        
    cartesians = [np.prod(np.sin(angles[:i]))*np.cos(angles[i]) for i in range(len(angles))]
    cartesians.append(np.prod(np.sin(angles)))
    vec = np.array(cartesians)
    return vec
    
def new_cost_F_rangle(vector):
    # vector = (q, r) = (q, angle)
    # r = (cos(angle), sin(angle))  
    # note vector here is size |G|+|C|-1
    
    if isinstance(vector, list):
        vector = np.array(vector)
    
    g_exp = contextual_sub.G_indices * vector[:contextual_sub.symmetry_generators.n_terms]
    g_exp[np.where(g_exp==0)]=1 # set values not affecting cost to 1 (so np.prod doesn't go to zero!)
    
    
#     angle = vector[-1]
#     r_vec = np.array([np.cos(angle),np.sin(angle)])
    angles = vector[contextual_sub.symmetry_generators.n_terms:]
    r_vec = unit_sphere(angles)
#     print(np.linalg.norm(r_vec))
    
    r_exp = contextual_sub.r_indices * r_vec
    r_exp[np.where(r_exp==0)]=1

    G = np.prod(g_exp, axis=1)
    R = np.prod(r_exp, axis=1)

#     return np.sum(contextual_sub.noncontextual_operator.coeff_vec*G*R).real
    return np.sum(contextual_sub.noncontextual_operator.coeff_vec.real*G*R)
    

In [405]:
c.shape

(74,)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1])

In [412]:

import numpy as np
from scipy.optimize import linprog
# https://docs.scipy.org/doc/scipy/reference/optimize.linprog-interior-point.html


# Set the inequality constraints matrix
# Note: the inequality constraints must be in the form of <=
G_size = contextual_sub.symmetry_generators.n_terms 
R_size = contextual_sub.n_cliques -1 
A_matrix = np.eye(G_size+R_size)

# lower bound 
G_lb = -1*np.ones(G_size)
R_lb = np.zeros(R_size)

b_lb = np.hstack((G_lb, R_lb))

# upper bound
G_ub = np.ones(G_size)
R_ub = np.array([2*np.pi for _ in range(R_size)])

b_ub = np.hstack((G_ub, R_ub))


c = contextual_sub.noncontextual_operator.coeff_vec.real

res = linprog(c, A_ub=np.eye(c.shape[0]),
              b_ub=np.abs(c),
              A_eq=np.eye(c.shape[0]),
              b_eq=-1*np.abs(c),
              bounds=None,method='interior-point')


res

     con: array([-9.00940166e+00, -2.24322924e+00, -2.24322924e+00, -5.68833460e-01,
       -4.66916923e-02, -1.31734600e-01, -1.42809077e-01, -4.66916923e-02,
       -1.42809077e-01, -1.31734600e-01, -1.01034405e-01, -1.48432476e-01,
       -1.38407439e-01, -1.42307209e-01, -8.06969811e-02, -1.02399226e-01,
       -1.48432476e-01, -1.42307209e-01, -1.38407439e-01, -1.02399226e-01,
       -8.06969811e-02, -1.12464760e-01, -1.48432476e-01, -1.38407439e-01,
       -1.42307209e-01, -8.06969811e-02, -1.02399226e-01, -9.42777259e-02,
       -1.00340071e-01, -1.48432476e-01, -1.42307209e-01, -1.38407439e-01,
       -1.02399226e-01, -8.06969811e-02, -1.00340071e-01, -9.42777259e-02,
       -1.12464760e-01, -1.48432476e-01, -1.38407439e-01, -1.42307209e-01,
       -8.06969811e-02, -1.02399226e-01, -9.42777259e-02, -1.00340071e-01,
       -9.42777259e-02, -1.00340071e-01, -1.48432476e-01, -1.42307209e-01,
       -1.38407439e-01, -1.02399226e-01, -8.06969811e-02, -1.00340071e-01,
       -9.42777

In [ ]:
# Set the inequality constraints matrix
# Note: the inequality constraints must be in the form of <=
G_size = contextual_sub.symmetry_generators.n_terms 
R_size = contextual_sub.n_cliques -1 
A_matrix = np.eye(G_size+R_size)

# lower bound 
G_lb = -1*np.ones(G_size)
R_lb = np.zeros(R_size)

b_lb = np.hstack((G_lb, R_lb))

# upper bound
G_ub = np.ones(G_size)
R_ub = np.array([2*np.pi for _ in range(R_size)])

b_ub = np.hstack((G_ub, R_ub))


c = contextual_sub.noncontextual_operator.coeff_vec.real

res = linprog(c, A_ub=A_matrix, b_ub=b_ub, A_eq=A_matrix, b_eq=b_lb, bounds=None,method='interior-point')

In [ ]:
# Form objective.
obj = cp.Minimize((x - y)**2)

# Form and solve problem.
prob = cp.Problem(obj, constraints)




In [353]:
size_G = contextual_sub.symmetry_generators.n_terms
size_R = contextual_sub.n_cliques
size = size_G + size_R
var_list = [cp.Variable() for var_ind in range(size)]

# G constraint
cons = [cp.square(var)<= 1 for var in var_list[:size_G]]

# R constraint
# cons += [cp.sum_squares(var_list[size_G:])==1]
cons += [cp.square(var_list[size_G:][0])+cp.square(var_list[size_G:][1])==1]

obj = cp.Minimize(new_cost_F(var_list))
prob = cp.Problem(obj, cons)
prob.solve()

/var/folders/7l/9g4tmq2s5zg48_szmd8z7qh40000gn/T/ipykernel_64757/2791763583.py:8: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  g_exp[np.where(g_exp==0)]=1 # set values not affecting cost to 1 (so np.prod doesn't go to zero!)
/var/folders/7l/9g4tmq2s5zg48_szmd8z7qh40000gn/T/ipykernel_64757/2791763583.py:11: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  r_exp[np.where(r_exp==0)]=1


DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
0.0 @ var9975 @ 0.0 @ var9976
0.0 @ var9984 @ 0.0 @ var9985
1.0 @ var9975 @ 0.0 @ var9976
0.0 @ var9984 @ 0.0 @ var9985
0.0 @ var9975 @ 0.0 @ var9976
1.0 @ var9984 @ 0.0 @ var9985
1.0 @ var9975 @ 0.0 @ var9976
1.0 @ var9984 @ 0.0 @ var9985
1.0 @ var9975 @ 1.0 @ var9976
0.0 @ var9984 @ 0.0 @ var9985
0.0 @ var9975 @ 1.0 @ var9976
0.0 @ var9984 @ 0.0 @ var9985
1.0 @ var9975 @ 1.0 @ var9976
1.0 @ var9984 @ 0.0 @ var9985
0.0 @ var9975 @ 0.0 @ var9976
1.0 @ var9984 @ 0.0 @ var9985
1.0 @ var9975 @ 0.0 @ var9976
1.0 @ var9984 @ 0.0 @ var9985
0.0 @ var9975 @ 0.0 @ var9976
0.0 @ var9984 @ 0.0 @ var9985
1.0 @ var9975 @ 1.0 @ var9976
1.0 @ var9984 @ 0.0 @ var9985
0.0 @ var9975 @ 0.0 @ var9976
0.0 @ var9984 @ 0.0 @ var9985
1.0 @ var9975 @ 0.0 @ var9976
0.0 @ var9984 @ 0.0 @ var9985
0.0 @ var9975 @ 0.0 @ var9976
1.0 @ var9984 @ 0.0 @ var9985
1.0 @ var9975 @ 1.0 @ var9976
0.0 @ v

In [201]:

# Create scalar optimization variables.
size = contextual_sub.symmetry_generators.n_terms + contextual_sub.n_cliques
var_list = = [cp.Variable() for var_ind in range(size)]

# Create two constraints.
constraints = [|var|<=1for var in var_list]

# Form objective.
obj = cp.Minimize((x - y)**2)

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()

# The optimal dual variable (Lagrange multiplier) for
# a constraint is stored in constraint.dual_value.
print("optimal (x + y == 1) dual variable", constraints[0].dual_value)
print("optimal (x - y >= 1) dual variable", constraints[1].dual_value)
print("x - y value:", (x - y).value)

0

In [197]:
optimizer_output.x

array([-1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        , -1.        ,
       -0.70649809])

In [184]:
angles = [np.pi, 2*np.pi]
cartesians = [np.prod(np.sin(angles[:i]))*np.cos(angles[i]) for i in range(len(angles))]
cartesians.append(np.prod(np.sin(angles)))
np.array(cartesians)

array([-1.00000000e+00,  1.22464680e-16, -2.99951957e-32])

In [ ]:
r_bounds = [(0, np.pi)]*(self.n_cliques-2)+[(0, 2*np.pi)]
optimizer_output = differential_evolution(
    func=lambda angles:self.noncontextual_objective_function(
        nu, unit_n_sphere_cartesian_coords(angles)
        ), 
    bounds=r_bounds

In [15]:
import cvxpy as cp

In [34]:
matrixSize =4
A = np.random.rand(matrixSize, matrixSize)
B = np.dot(A, A.transpose())

B

array([[1.10427193, 0.99040005, 1.1384572 , 0.6317542 ],
       [0.99040005, 1.33268412, 1.21742497, 0.60147133],
       [1.1384572 , 1.21742497, 1.80624826, 0.4064872 ],
       [0.6317542 , 0.60147133, 0.4064872 , 0.74301387]])

In [58]:
(1+1)%2

0

In [59]:
# brute force
num_G = contextual_sub.symmetry_generators.n_terms
num_A = contextual_sub.n_cliques
opt_vec_size = num_G + num_A

X = cp.Variable((opt_vec_size))

constraints = [(1+X[ind])%2==0 for ind in range(num_G)],

constraints += [np.linalg.norm(X[contextual_sub.symmetry_generators.n_terms:]==1)]


TypeError: unsupported operand type(s) for %: 'AddExpression' and 'int'

In [168]:
n_gen=3
G_matrix = np.random.rand(n_gen, n_gen)* np.random.choice([-1,1], size=(n_gen, n_gen))
denom = np.sqrt(np.einsum('ij->i',np.abs(G_matrix)**2))

matrix = G_matrix / denom.reshape((n_gen,1))

# sum(matrix[0]**2)
np.einsum('ij->i', matrix**2)

array([1., 1., 1.])

In [172]:
matrix[0].dot(matrix[])

-0.2068582449457306

In [159]:
G_matrix/denom.reshape((n_gen,1))



array([1., 1., 1.])

1.1318401121306578

In [143]:
G_matrix**2

array([[0.19680819, 0.02129569, 0.01688673],
       [0.71889439, 0.21723831, 0.61067591],
       [0.70460741, 0.84595524, 0.92850381]])

In [105]:
# n_gen = contextual_sub.symmetry_generators.n_terms 
# g_vec = np.random.rand(n_gen) * np.random.choice([-1,1], size=n_gen)
# # g_vec_norm = g_vec / np.linalg.norm(g_vec)

In [ ]:
opt_vec_size = contextual_sub.symmetry_generators.n_terms + contextual_sub.n_cliques

X = cp.Variable((opt_vec_size,opt_vec_size), symmetric=True)

constraints = [[X>>0],
              [X[i,i]==1 for i in range(opt_vec_size)]]

In [255]:
angles = np.array([1,2,3])

cartesians = [np.prod(np.sin(angles[:i]))*np.cos(angles[i]) for i in range(len(angles))]
cartesians.append(np.prod(np.sin(angles)))
print(np.linalg.norm(cartesians))

cartesians

1.0


[0.5403023058681398,
 -0.35017548837401463,
 -0.7574901860152801,
 0.10797760742916983]

In [435]:
A = np.array([[1],[2],[3]])

B = np.array([[1,2,3],
             [4,5,6],
             [7,8,9]])

B@A

array([[14],
       [32],
       [50]])